In [108]:
# Load libraries
import pandas as pd
from dash import Dash, dcc, html, Input, Output, callback
import sqlite3
import plotly.express as px
import plotly
import lxml
import numpy as np
import os

In [ ]:
# Process history.sqlite Data

# Import data
con = sqlite3.connect("C:/Data_analysis/timsQC/history.sqlite")
cur = con.cursor()
data = pd.read_sql_query(f"SELECT * FROM InstrumentParameterHistory", con)

data[['Date', 'Time']] = data['DateTime'].str.split('T', expand=True)
# MZ cal
mzCal = data[data['Comment'].str.contains("Calibration of type 'm/z'")]
mzCal[['score']] = mzCal['Comment'].str.extract(fr'{'score'}\s+(.*?)\s+{'and'}')
mzCal['score'] = mzCal['score'].str.strip("'")
mzCal['score'] = mzCal['score'].str.replace("%","",regex=False)
mzCal['score'] = pd.to_numeric(mzCal['score'], errors='coerce')
mzCal[['relative_deviation']] = mzCal['Comment'].str.extract(fr'{'deviation'}\s+(.*?)\s+')
mzCal['relative_deviation'] = mzCal['relative_deviation'].str.strip("'")
mzCal['relative_deviation'] = pd.to_numeric(mzCal['relative_deviation'], errors='coerce')

# Mobility
mobCal = data[data['Comment'].str.contains("Calibration of type 'mobility'")]
mobCal[['score']] = mobCal['Comment'].str.extract(fr'{'score'}\s+(.*?)\s+{'and'}')
mobCal['score'] = mobCal['score'].str.strip("'")
mobCal['score'] = mobCal['score'].str.replace("%","",regex=False)
mobCal['score'] = pd.to_numeric(mobCal['score'], errors='coerce')
mobCal[['relative_deviation']] = mobCal['Comment'].str.extract(fr'{'deviation'}\s+(.*?)\s+')
mobCal['relative_deviation'] = mobCal['relative_deviation'].str.strip("'")
mobCal['relative_deviation'] = pd.to_numeric(mobCal['relative_deviation'], errors='coerce')

# ToF Tuning
tofTuning = data[data['Comment'].str.contains("TofDetectorTuning")]
# Cap cleaning
capCleaning = data[data['Comment'].str.contains("Update capillary cleaning date")]

# Make data wide for heatmaps
mobCal_wide = mobCal.pivot(columns='Date', index='Time', values='score')

In [81]:
# file list walker function

def get_files_one_level_deep(directory, file_extension):
    """
    Gets a list of files with a specific extension within a directory,
    recursively, but only one level deep.

    Args:
        directory (str): The path to the directory to search.
        file_extension (str): The file extension to look for (e.g., ".txt").

    Returns:
        list: A list of file paths that match the criteria.
    """
    matching_files = []
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isfile(item_path) and item.lower().endswith(file_extension):
            matching_files.append(item_path)
        elif os.path.isdir(item_path):
            for sub_item in os.listdir(item_path):
              sub_item_path = os.path.join(item_path, sub_item)
              if os.path.isfile(sub_item_path) and sub_item.lower().endswith(file_extension):
                  matching_files.append(sub_item_path)
    return matching_files

In [ ]:
# XML importer for ISO-8859-1

import xml.etree.ElementTree as ET

def parse_xml_iso_8859_1(filename):
    """Parses an XML file with ISO-8859-1 encoding."""
    try:
        tree = ET.parse(filename)
        return tree
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
xml_file = "../new_microTOFQImpacTemAcquisition.method"  # Replace with your file path
tree_element = parse_xml_iso_8859_1(xml_file)
root_element = tree_element.getroot()

In [208]:
# Imports from .method file

target_directory = "../"  # Replace with your directory path
target_extension = ".method"  # Replace with your desired file extension
methList = get_files_one_level_deep(target_directory, target_extension)

massCalMaster=pd.DataFrame()
massCalMaster_wide=pd.DataFrame()
mobilityCalMaster=pd.DataFrame()


for i in methList:
    print(i)
    tree_element = parse_xml_iso_8859_1(i)
    root_element = tree_element.getroot()
    tree=tree_element
    root=root_element
    CalLists = []
    CalVals = []
# transfers XML structure to dataframes for positive calibrations ONLY
    for container in root.findall('instrument/qtofimpactemacq/'):
        for element in container.iter('dependent'):
            if element.get('polarity')=='positive':
                # numbers (para_double)
                for item in element.findall('para_double'):
                    child_dict = {}
                    child_dict['tag'] = item.get('permname')
                    child_dict['value'] = item.get('value')
                    if item.text:
                        child_dict['text'] = item.text.strip()
                    CalVals.append(child_dict)
                # numbers (para_int)
                for item in element.findall('para_int'):
                    child_dict = {}
                    child_dict['tag'] = item.get('permname')
                    child_dict['value'] = item.get('value')
                    if item.text:
                        child_dict['text'] = item.text.strip()
                    CalVals.append(child_dict)
                # strings (para_string)
                for item in element.findall('para_string'):
                    child_dict = {}
                    child_dict['tag'] = item.get('permname')
                    child_dict['value'] = item.get('value')
                    if item.text:
                        child_dict['text'] = item.text.strip()
                    CalVals.append(child_dict)
                # lists (para_vec_double)
                for item in element.findall('para_vec_double'):
                    for child in item:
                        child_dict = {}
                        child_dict['tag'] = item.get('permname')
                        child_dict['value'] = child.get('value')
                        if child.text:
                            child_dict['text'] = child.text.strip()
                        CalVals.append(child_dict)
    df_CalVals=pd.json_normalize(CalVals) # single values
    # write raw file 
    filename=i.replace("../", "")
    df_CalVals.to_csv(f'C:/Data_analysis/timsQC/timsCal/{filename}.csv', header=None, index=None, sep=',', mode='w')
        
# Separation
    
    massCal_df=df_CalVals[df_CalVals['tag'].isin(['Calibration_LastCalibrationDate',
                                            'Calibration_Score',
                                            'Calibration_StdDev',
                                            'Calibration_StdDevInPPM',
                                            'Calibration_LastCalibrationCurrentMass'])]

    massCalFrame_df=df_CalVals[df_CalVals['tag'].isin(['Calibration_LastCalibrationCurrentMass',
                                                    'Calibration_LastCalibrationMassError',
                                                    'Calibration_LastCalibrationMassIntensity'])]
    massCalFrame_df['tag'] = massCalFrame_df.apply(lambda x: x['tag'].replace('Calibration_LastCalibration', ''), axis=1)
    massCalFrame_df.insert(1, 'seq', massCalFrame_df.groupby('tag').cumcount() + 1)
    calDate=df_CalVals[df_CalVals['tag'].isin(['Calibration_LastCalibrationDate'])]['value']
    calDate=calDate.tolist()
    massCalFrame_df=massCalFrame_df.assign(date=calDate[0])
    massCalFrame_wide_df=massCalFrame_df.pivot(index='seq', columns='tag', values='value')
    massCalFrame_wide_df=massCalFrame_wide_df.assign(date=calDate[0])


    mobilityCal_df=df_CalVals[df_CalVals['tag'].isin(['IMS_Calibration_LastCalibrationDate',
                                                    'IMS_Calibration_LastCalibrationReferenceMassList',
                                                    'IMS_Calibration_Score',
                                                    'IMS_Calibration_StdDev'])]

    mobilityCalFrame_df=df_CalVals[df_CalVals['tag'].isin(['IMS_Calibration_LastCalibrationReferenceMass',
                                                    'IMS_Calibration_LastCalibrationReferenceMobility',
                                                    'IMS_Calibration_LastCalibrationResultMobility',
                                                    'IMS_Calibration_LastCalibrationMassIntensity'])]
    mobilityCalFrame_df['tag'] = mobilityCalFrame_df.apply(lambda x: x['tag'].replace('IMS_Calibration_LastCalibration', ''), axis=1)
    mobilityCalFrame_df.insert(1, 'seq', mobilityCalFrame_df.groupby('tag').cumcount() + 1)
    calDate=df_CalVals[df_CalVals['tag'].isin(['IMS_Calibration_LastCalibrationDate'])]['value']
    calDate=calDate.tolist()
    mobilityCalFrame_df=mobilityCalFrame_df.assign(date=calDate[0])
    mobilityCalFrame_wide_df=mobilityCalFrame_df.pivot(index='seq', columns='tag', values='value')
    mobilityCalFrame_wide_df=mobilityCalFrame_wide_df.assign(date=calDate[0])

    tofCal_df=df_CalVals[df_CalVals['tag'].isin(['TOF_DetectorTof_LastCalibrationDate',
                                                'Calibration_Tof2Score',
                                                'Calibration_Tof2StdDev',
                                                'Calibration_Tof2StdDevInPPM'])]

    quadCal=df_CalVals[df_CalVals['tag'].isin(['Quadrupole_Ramping_LastCalibrationDate'])]

    massCalMaster=pd.concat([massCalMaster, massCalFrame_df])
    massCalMaster_wide=pd.concat([massCalMaster_wide, massCalFrame_wide_df])


../new_microTOFQImpacTemAcquisition.method
../old_microTOFQImpacTemAcquisition.method


C:\Users\MRGBRA003\AppData\Local\Temp\ipykernel_12512\3105667012.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\MRGBRA003\AppData\Local\Temp\ipykernel_12512\3105667012.py:91: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\MRGBRA003\AppData\Local\Temp\ipykernel_12512\3105667012.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [209]:
# reformatting

massCalMaster_wide['CurrentMass']=massCalMaster_wide['CurrentMass'].astype('float')
massCalMaster_wide['MassError']=massCalMaster_wide['MassError'].astype('float')
massCalMaster_wide['MassIntensity']=massCalMaster_wide['MassIntensity'].astype('float')
massCalMaster['value']=massCalMaster['value'].astype('float')



In [ ]:
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
import pandas as pd

fig = px.scatter(massCalMaster_wide, x='CurrentMass', y='MassIntensity', color='date', title='Time Series Plot')

app = Dash(__name__)

app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

if __name__ == '__main__':
    app.run(debug=True)

# Internal logic of calibration parameter value selection


In [ ]:
# App Layout

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

#df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
df=massCalFrame_df

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['tag'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['tag'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                xaxis_type, yaxis_type,
                year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['tag'] == xaxis_column_name]['Value'],
            y=dff[dff['tag'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['tag'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['tag'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                    xref='paper', yref='paper', showarrow=False, align='left',
                    text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['tag'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['tag'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run(debug=True)

KeyError: 'Year'

In [ ]:
# Params

"LastCalibrationDate" # calibration date (any)

"Calibration_LastCalibrationDate" # mass cal date
"Calibration_Score" # mass cal score
"Calibration_StdDev" # mass cal deviation
"Calibration_StdDevInPPM" # mass cal deviation in PPM

"IMS_Calibration_LastCalibrationDate" # mobility cal date
"IMS_Calibration_Score" # mobility cal score
"IMS_Calibration_StdDev" # mobility cal deviation

"TOF_DetectorTof_LastCalibrationDate" # ToF cal date
"Calibration_Tof2Score",
"Calibration_Tof2StdDev",
"Calibration_Tof2StdDevInPPM",

"Quadrupole_Ramping_LastCalibrationDate" # quad ramping cal date

# unassigned
"Calibration_LastCalibrationCurrentMass.entry_double",
"Calibration_LastCalibrationMassError.entry_double",
"Calibration_LastCalibrationMassIntensity.entry_double",

"IMS_Calibration_LastCalibrationReferenceMass",
"IMS_Calibration_LastCalibrationMobilityError",
"IMS_Calibration_LastCalibrationMassIntensity",
"IMS_Calibration_LastCalibrationReferenceMassList",
"IMS_Calibration_LastCalibrationReferenceMobility",
"IMS_Calibration_LastCalibrationResultMobility",

"Calibration_TOF_CorrectorExtractSetValue",
"Calibration_TOF_CorrectorFillSetValue",
"Calibration_TOF_DeceleratorSetValue",
"Calibration_TOF_DetectorTofSetValue",
"Calibration_TOF_ReflectorSetValue",
"Calibration_TOF_FlightTubeSetValue",
"TOF_DeviceReferenceTemp1" # TOF temp 1
"TOF_DeviceReferenceTemp2"  # TOF temp 2
"Calibration_Tof2CalC0",
"Calibration_Tof2CalC1",
"Calibration_Tof2CalC2",
"Calibration_TofCalC0",
"Calibration_TofCalC1",
"Calibration_TofCalC2",
"Calibration_Collision_GasSupply_Set",

"Vacuum_LastCapillaryCleaningDate"

In [ ]:
# # TEMP

# tree=tree_element
# root=root_element

# CalLists = []
# CalVals = []

# # transfers XML structure to dataframes
# for container in root.findall('instrument/qtofimpactemacq/'):
#     for element in container.iter('dependent'):
#         # numbers (para_double)
#         for item in element.findall('para_double'):
#             child_dict['tag'] = item.get('permname')
#             child_dict['value'] = item.get('value')
#             #print('attrib')
#             #print(child.attrib)
#             if child.text:
#                 child_dict['text'] = child.text.strip()
#             #val_dict['text'] = child.text.strip()
#             CalVals.append(child_dict)
#         # numbers (para_int)
#         for item in element.findall('para_int'):
#             child_dict = {}
#             child_dict['tag'] = item.get('permname')
#             child_dict['value'] = item.get('value')
#             #print('attrib')
#             #print(child.attrib)
#             if child.text:
#                 child_dict['text'] = child.text.strip()
#             #val_dict['text'] = child.text.strip()
#             CalVals.append(child_dict)
#         # strings (para_string)
#         for item in element.findall('para_string'):
#             child_dict = {}
#             child_dict['tag'] = item.get('permname')
#             child_dict['value'] = item.get('value')
#             #print('attrib')
#             #print(child.attrib)
#             if child.text:
#                 child_dict['text'] = child.text.strip()
#             #val_dict['text'] = child.text.strip()
#             CalVals.append(child_dict)
#         # lists (para_vec_double)
#         for item in element.findall('para_vec_double'):
#             for child in item:
#                 child_dict = {}
#                 child_dict['parameter'] = item.get('permname')
#                 #child_dict['tag'] = child.get('entry_double')
#                 child_dict['value'] = child.get('value')
#                 #child_dict['attributes'] = child.attrib
#                 #child_dict['parent'] = element.attrib
#                 #print('attrib')
#                 #print(child.attrib)
#                 if child.text:
#                     child_dict['text'] = child.text.strip()
#                 CalLists.append(child_dict)
#                 #val_dict = {}
#                 #val_dict['tag'] = item.get('permname')
#                 #val_dict['value'] = item.get('value')
#                 #val_dict['attributes'] = child.attrib
#                 #val_dict['parent'] = element.attrib
#                 #val_dict['text'] = child.text.strip()
#                 #list_of_dicts.append(val_dict)
# df_CalLists=pd.json_normalize(CalLists) # lists from XML
# df_CalVals=pd.json_normalize(CalVals) # single values

In [ ]:
# calplot code temp

scoreplot = calplot(mobCal, x='Date', y="score",  years_title=True, dark_theme=False, showscale = True)
deviationplot = calplot(mobCal, x='Date', y="relative_deviation",  years_title=True, dark_theme=False, showscale = True)
mzscoreplot = calplot(mzCal, x='Date', y="score",  years_title=True, dark_theme=False, showscale = True)
mzdeviationplot = calplot(mzCal, x='Date', y="relative_deviation",  years_title=True, dark_theme=False, showscale = True)

# main
app.layout = html.Div([
    html.H1(children="TIMS ToF Analytics"),
    html.H4('Mobility Scores'),
    dcc.Graph(figure=scoreplot),
    html.H4('Mobility Relative Deviation'),
    dcc.Graph(figure=deviationplot),
    html.H4('MZ Scores'),
    dcc.Graph(figure=mzscoreplot),
    html.H4('MZ Relative Deviation'),
    dcc.Graph(figure=mzdeviationplot)
])